Problema based on an exercise taken from Luyben and Hill

 Author
 Prof. Ali Gabriel Lara

 April 2014
 Revision: None

In [1]:
import numpy as np
import math
from scipy.optimize import fsolve, fmin

In [2]:
def diameterfun(dguess, params, fun_out=False):
    vr, dcoil = params
    lvessel = 2 * dguess
    nloops = math.ceil(lvessel/2/dcoil)
    lcoil = 0.8 * dguess * np.pi * nloops
    vcoil = lcoil * np.pi * (dcoil**2)/4
    vtot = vr + vcoil
    dvessel = (2 * vtot/np.pi)**0.3333
    if fun_out:
        return [lcoil, nloops, vtot, vcoil]
    else:
        return dvessel - dguess

def temperaturefun(tcout, params, fun_out=False):
    tcin, tr, acoil, u, qr = params

    delta_temp_in = tr - tcin
    delta_temp_out = tr - tcout
    dtml = (delta_temp_in-delta_temp_out) / np.log((delta_temp_in)/abs(delta_temp_out))
    qcal = u * acoil * dtml
    if fun_out:
        return dtml
    else:
        return abs(qcal - qr)

In [3]:
Tcin = 343                 # K
U = 100                    # W/m^2-K
Cpj = 4183                  # J/kg-K
Dcoil = 0.015875           # m

Tr = 163 + 273.15           # K
Vr = 5.825132               # m^3
Qr = 1.1687e7/3600*4.1868   # J/s

# Internal coil diameter

In [4]:
dguess = (2 * Vr / np.pi)**0.333

parameters = [Vr, Dcoil]
Dr = fsolve(lambda x: diameterfun(x, parameters), dguess)

In [5]:
[Lcoil, Nloops, Vtot, Vcoil] = diameterfun(Dr[0], parameters, fun_out=True)
Acoil = np.pi*Dcoil*Lcoil

In [6]:
print('Reactor size parameters including coil heat exchanger:')
print('Reactor diameter: {:1.2f} m'.format(Dr[0]))
print('Reactor volume: {:1.1f} m'.format(math.ceil(Vtot)))
print('\n')

print('Coil heat exchanger parameters:')
print('Transfer area: {:1.2f} m^2'.format(Acoil))
print('Number of loops: {:1.0f}'.format(Nloops))
print('Coil volume: {:1.2f} m^3'.format(Vcoil))
print('Coil length: {:1.2f} m'.format(Lcoil))

Reactor size parameters including coil heat exchanger:
Reactor diameter: 1.55 m
Reactor volume: 6.0 m


Coil heat exchanger parameters:
Transfer area: 19.09 m^2
Number of loops: 98
Coil volume: 0.08 m^3
Coil length: 382.87 m


# Output temperature of the heat-exchanger fluid

In [7]:
Tcout_seed = 0.9*Tr
delta_temp_in = Tr - Tcin
delta_temp_out = Tr - Tcout_seed

In [8]:
Tcout_seed = 300

parameters = [Tcin, Tr, Acoil, U, Qr]
Tcout = fmin(lambda x: temperaturefun(x, parameters), Tcout_seed, ftol=0.001)

Optimization terminated successfully.
         Current function value: 0.002211
         Iterations: 41
         Function evaluations: 83


In [9]:
Fcoil = Qr/(Cpj*(Tcout[0]-Tcin))
Fcoil = Fcoil/0.001

DTLM = temperaturefun(Tcout[0], parameters, fun_out=True)

In [10]:
print('Cooling fluid parameters:')
print('Exit temperature : {:1.2f} K'.format(Tcout[0]))
print('Log-mean temperature difference : {:1.2f} K'.format(DTLM))
print('Coolant flowrate : {:1.2f} cm^3/s'.format(Fcoil))

Cooling fluid parameters:
Exit temperature : 436.15 K
Log-mean temperature difference : 7.12 K
Coolant flowrate : 34.88 cm^3/s
